In [1]:
import pandas as pd

In [8]:
df = pd.read_csv('data.csv',index_col=0)

In [3]:
import gensim.models

In [4]:
model = gensim.models.Word2Vec.load('word2vec.fullmodel')

In [6]:
len(model.wv)

211464

In [10]:
captions=list(df['caption'])

In [12]:
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string,strip_non_alphanum,strip_numeric

In [18]:
for line in open(f'{captions[0]}'):
    CUSTOM_FILTERS = [lambda x: x.lower(), remove_stopwords, strip_non_alphanum,strip_numeric]
    tokens=preprocess_string(line,CUSTOM_FILTERS)
    tokens_filtered_in_vocab = [token for token in tokens if token in model.wv.index_to_key] # Filter out out of vocab tokens

In [20]:
import numpy as np

-2.0413594484329223

In [38]:
embedding = embedding - min(embedding)

In [42]:
embedding = embedding / max(embedding)

In [46]:
out_string

',0.45350155971180706,0.3800610459612754,0.3849763393052144,0.4705067313420381,0.7200443415374759,0.9018688458171419,0.4514048428742117,0.43210801174895436,0.686250401161629,0.47221142328592264,0.630813467100926,0.895093776698023,0.6617422366222705,0.23865608963107246,0.3940583823045318,0.5626082162489685,0.40785245677608467,0.8340910643014352,0.7785953289859912,0.3822970511166835,0.6582121043157304,0.5636783254503643,0.4314199464323862,0.2964874136299024,0.6410295859583822,0.6108022948250351,0.5292383822619953,0.37298645503169653,0.5864829065402798,0.6911801236914589,0.2987429101249909,0.9106751308152344,0.59617363161106,0.8051101855894898,0.517831534792387,0.7721364502025226,0.6935711666971768,0.7159768414209182,0.30862786974602785,0.7265715974947003,0.8403294525560068,0.7603411436646158,0.5481821628127233,0.5099791662997941,0.5266046691720943,0.5722608556876108,0.5697325421978203,0.5484527140943732,0.7218214836298965,0.4317554613221428,0.7146369973093666,0.7070353758790385,0.6674453

In [58]:
def caption_2_represention(captionfile):
    size=300
    for line in open(f'{captions[0]}'):
        CUSTOM_FILTERS = [lambda x: x.lower(), remove_stopwords, strip_non_alphanum,strip_numeric]
        tokens=preprocess_string(line,CUSTOM_FILTERS)
        tokens_filtered_in_vocab = [token for token in tokens if token in model.wv.index_to_key] 
        embedding = np.zeros(size)
        for tok in tokens_filtered_in_vocab:
            embedding += model.wv[tok]
            
        if len(tokens_filtered_in_vocab) > 1:
            embedding /= len(tokens_filtered_in_vocab)
            
        if min(embedding) < 0:
            embedding = embedding - min(embedding)
        if max(embedding) > 0:
            embedding = embedding / max(embedding)
        out_string = ''
        for t in range(0,300):
            out_string = out_string + ','+ str(embedding[t])
        return out_string

In [59]:
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [60]:
train_df = pd.read_csv('train.csv',index_col=0)

In [61]:
train_df.head()

,caption,img,label
656029,captions_resized_1M/cities_instagram/chicago/1...,img_resized_1M/cities_instagram/chicago/148202...,chicago
595442,captions_resized_1M/cities_instagram/miami/148...,img_resized_1M/cities_instagram/miami/14898629...,miami
245192,captions_resized_1M/cities_instagram/london/14...,img_resized_1M/cities_instagram/london/1490670...,london
841946,captions_resized_1M/cities_instagram/losangele...,img_resized_1M/cities_instagram/losangeles/149...,losangeles
302005,captions_resized_1M/cities_instagram/sydney/14...,img_resized_1M/cities_instagram/sydney/1480253...,sydney


In [62]:
train_df['caption_embedding'] = train_df['caption'].parallel_apply(caption_2_represention)

In [63]:
train_df.head()

,caption,img,label,caption_embedding
656029,captions_resized_1M/cities_instagram/chicago/1...,img_resized_1M/cities_instagram/chicago/148202...,chicago,",0.45350155971180706,0.3800610459612754,0.3849..."
595442,captions_resized_1M/cities_instagram/miami/148...,img_resized_1M/cities_instagram/miami/14898629...,miami,",0.45350155971180706,0.3800610459612754,0.3849..."
245192,captions_resized_1M/cities_instagram/london/14...,img_resized_1M/cities_instagram/london/1490670...,london,",0.45350155971180706,0.3800610459612754,0.3849..."
841946,captions_resized_1M/cities_instagram/losangele...,img_resized_1M/cities_instagram/losangeles/149...,losangeles,",0.45350155971180706,0.3800610459612754,0.3849..."
302005,captions_resized_1M/cities_instagram/sydney/14...,img_resized_1M/cities_instagram/sydney/1480253...,sydney,",0.45350155971180706,0.3800610459612754,0.3849..."


In [65]:
train_df.to_csv('train.csv',index=False)

In [66]:
test_df = pd.read_csv('test.csv',index_col=0)

In [67]:
test_df['caption_embedding'] = test_df['caption'].parallel_apply(caption_2_represention)

In [68]:
test_df.to_csv('test.csv',index=False)

In [69]:
val_df = pd.read_csv('val.csv',index_col=0)

In [70]:
val_df['caption_embedding'] = val_df['caption'].parallel_apply(caption_2_represention)

In [71]:
val_df.to_csv('val.csv',index=False)